In [1]:
# GENERAL THINGS FOR COMPUTING AND PLOTTING
import pandas as pd
import numpy as np
import os, sys, time
from datetime import datetime
from datetime import timedelta
import scipy as sp

# visualisation
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="ticks", context="talk")

# ibl specific things
import datajoint as dj
from ibl_pipeline import subject, acquisition

## get the latest training status for each mouse
from ibl_pipeline.analyses import behavior
pd.set_option('display.max_rows', 1000)


Connecting anneurai@datajoint.internationalbrainlab.org:3306
Connected to https://alyx.internationalbrainlab.org as anneu


In [37]:
behavior.SessionTrainingStatus()

subject_uuid,session_start_time start time,training_status,good_enough_for_brainwide_map
00778394-c956-408d-8a6c-ca3b05a611d5,2019-08-10 11:24:59,in_training,0
00778394-c956-408d-8a6c-ca3b05a611d5,2019-08-12 09:21:03,in_training,0
00778394-c956-408d-8a6c-ca3b05a611d5,2019-08-13 10:28:45,in_training,0
00778394-c956-408d-8a6c-ca3b05a611d5,2019-08-14 09:37:17,in_training,0
00778394-c956-408d-8a6c-ca3b05a611d5,2019-08-14 11:35:16,in_training,0
00778394-c956-408d-8a6c-ca3b05a611d5,2019-08-15 10:29:21,in_training,0
00778394-c956-408d-8a6c-ca3b05a611d5,2019-08-16 17:00:11,in_training,0
00778394-c956-408d-8a6c-ca3b05a611d5,2019-08-19 09:12:02,in_training,0
00778394-c956-408d-8a6c-ca3b05a611d5,2019-08-20 12:06:48,in_training,0
00778394-c956-408d-8a6c-ca3b05a611d5,2019-08-21 11:21:03,in_training,0


In [43]:
subj_query = ((subject.Subject & 'subject_nickname NOT LIKE "%human%"') - subject.Death) * \
subject.Subject.aggr(behavior.SessionTrainingStatus, session_start_time='max(session_start_time)') * \
behavior.SessionTrainingStatus.proj('training_status', last_session='DATE(session_start_time)') * subject.SubjectLab
df = subj_query.fetch(format='frame').reset_index()
df

,subject_uuid,session_start_time,subject_nickname,sex,subject_birth_date,ear_mark,subject_line,subject_source,protocol_number,subject_description,subject_ts,subject_strain,training_status,last_session,lab_name,subjectlab_ts
0,069f108c-c144-40c1-953d-2502950de79d,2019-10-15 15:57:29,CSK-scan-007,M,2019-06-13,None,VGAT-ChR2,CSHL-Harris,3,clear skull cap for inhibition scan in IBL task,2019-08-30 04:24:54,B6.Cg,trained_1a,2019-10-15,zadorlab,2020-09-02 14:03:17
1,0812d02d-d62b-416d-8b66-7956db34ee32,2020-07-24 17:54:50,ibl_witten_19,M,2019-11-13,N/A,C57BL/6J,Jax,3,None,2020-02-07 05:20:40,C57BL/6J,ready4recording,2020-07-24,wittenlab,2020-09-02 14:03:17
2,0bde9528-5bed-4ba5-a95a-a250cd769bbe,2020-09-03 14:25:35,SWC_029,F,2019-08-16,R,C57BL/6J,Charles River,1,None,2019-11-07 04:38:00,C57BL/6J,trained_1a,2020-09-03,mrsicflogellab,2020-09-02 14:03:18
3,0d7d50d7-41c4-4bb9-bf67-d51f0dc83faa,2020-09-01 13:30:00,ibl_witten_23,F,2019-12-03,NA,C57BL/6J,Jax,3,None,2020-02-29 04:38:41,C57BL/6J,trained_1b,2020-09-01,wittenlab,2020-09-02 14:03:18
4,0e54d7ec-92c5-4cc0-bb71-ca500f20e4b4,2020-09-01 11:28:21,NYU-28,M,2020-05-19,None,None,None,1,None,2020-07-14 04:48:09,None,trained_1a,2020-09-01,angelakilab,2020-09-02 14:03:18
5,0f0f8900-194e-4333-acac-9ff11bcc354b,2020-03-23 11:21:45,CSK-scan-014,M,2019-11-03,RL,VGAT-ChR2,CSHL-Harris,3,for scanning inhibition,2020-01-24 05:05:32,C57BL/6J,trained_1b,2020-03-23,zadorlab,2020-09-02 14:03:18
6,10a1fd99-5d9d-4a64-827b-f198627e330f,2019-10-28 16:53:59,CSK-scan-006,M,2019-06-13,R,VGAT-ChR2,CSHL-Harris,3,clear skull prep for inhibition scanning in IB...,2019-08-29 04:24:21,B6.Cg,trained_1b,2019-10-28,zadorlab,2020-09-02 14:03:18
7,12385eb1-eb86-496d-91c6-a4bd7392376a,2020-08-28 12:51:51,ibl_witten_18,M,2019-11-13,N/A,C57BL/6J,Jax,3,None,2020-02-07 05:20:40,C57BL/6J,ready4delay,2020-08-28,wittenlab,2020-09-02 14:03:18
8,16c81b7c-4a93-4062-8d5e-8eb01f42035d,2020-08-31 08:37:22,NR_0005,M,2020-05-10,None,None,None,3,None,2020-07-25 04:55:20,None,in_training,2020-08-31,steinmetzlab,2020-09-02 14:03:19
9,17715189-8c56-4c3a-81c6-eec2f68bbf10,2020-09-01 10:52:20,NYU-23,M,2020-01-07,None,C57BL/6J,Jax,1,None,2020-03-07 04:51:56,C57BL/6J,trained_1b,2020-09-01,angelakilab,2020-09-02 14:03:19


In [45]:
df['status'] = df['training_status'] + '; ' + (df['last_session']).astype(str)
df.groupby(['lab_name', 'subject_nickname'])['status'].unique()

lab_name        subject_nickname
angelakilab     CSP012                   [trained_1a; 2020-09-01]
                CSP013                   [trained_1b; 2020-09-01]
                CSP015                  [in_training; 2020-09-01]
                CSP016                   [trained_1a; 2020-09-01]
                NYU-23                   [trained_1b; 2020-09-01]
                NYU-25                   [trained_1a; 2020-09-01]
                NYU-27                  [in_training; 2020-09-01]
                NYU-28                   [trained_1a; 2020-09-01]
                NYU-29                  [ready4delay; 2020-09-01]
                NYU-30                   [trained_1b; 2020-09-01]
                NYU-31                  [in_training; 2020-09-01]
                NYU-32                  [in_training; 2020-09-01]
                SH016                   [in_training; 2020-09-01]
                SH017                    [trained_1b; 2020-09-01]
churchlandlab   CSHL024                  [t